Google Cloud authentication and data download

In [1]:
# ! gcloud auth login
# ! gcloud auth application-default login
# ! gcloud config set project dl4h-final-project-383605

In [3]:
# ! pip install --upgrade google-api-python-client google-cloud-storage
from google.cloud import storage

# Replace these values with your project and bucket as needed
project_id = "dl4h-final-project-383605"
mimic3_bucket = "mimiciii-1.4.physionet.org"

storage_client = storage.Client(project=project_id)
bucket = storage_client.bucket(mimic3_bucket)
data_folder = "./data"
for blob in bucket.list_blobs():
  if "CHARTEVENTS" in blob.name:
    continue
  blob.download_to_filename(f"{data_folder}/{blob.name}")

# Extract all the files
! gunzip {data_folder}/*.gz

gzip: *.gz: No such file or directory


In [5]:
! gunzip {data_folder}/*.gz

Library imports and data preparation

In [85]:
# ! pip install pyhealth
from pyhealth.datasets import MIMIC3Dataset, SampleDataset
from pyhealth.data import Visit
import pandas as pd
from pyhealth.datasets import split_by_patient, get_dataloader
from pyhealth.models import BaseModel
from pyhealth.trainer import Trainer
from pyhealth.metrics.binary import binary_metrics_fn
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List, Dict, Optional
from enum import Enum
from functools import reduce
from operator import mul

# Set this to the directory with all MIMIC-3 dataset files
data_root = "./data"

In [92]:
# Load the dataset

mimic3_ds = MIMIC3Dataset(
        root=data_root,
        tables=["DIAGNOSES_ICD", "PROCEDURES_ICD"],
        dev=False
)

In [93]:
# Print dataset statistics

mimic3_ds.stat()


Statistics of base dataset (dev=False):
	- Dataset: MIMIC3Dataset
	- Number of patients: 46520
	- Number of visits: 58976
	- Number of visits per patient: 1.2678
	- Number of events per visit in DIAGNOSES_ICD: 11.0384
	- Number of events per visit in PROCEDURES_ICD: 4.0711



'\nStatistics of base dataset (dev=False):\n\t- Dataset: MIMIC3Dataset\n\t- Number of patients: 46520\n\t- Number of visits: 58976\n\t- Number of visits per patient: 1.2678\n\t- Number of events per visit in DIAGNOSES_ICD: 11.0384\n\t- Number of events per visit in PROCEDURES_ICD: 4.0711\n'

In [94]:
# Find all diagnoses codes
# Find all procedure codes
# Remove diagnoses codes with fewer than 5 occurences in the dataset

all_diag_codes = []
for patient_id, patient in mimic3_ds.patients.items():
  for i in range(len(patient)):
    visit: Visit = patient[i]
    conditions = visit.get_code_list(table="DIAGNOSES_ICD")
    all_diag_codes.extend(conditions)

codes = pd.Series(all_diag_codes)
diag_code_counts = codes.value_counts()
filtered_diag_codes = diag_code_counts[diag_code_counts > 4].index.values
num_unique_diag_codes = len(filtered_diag_codes)

In [190]:
# Define the tasks

DIAGNOSES_KEY = "conditions"
PROCEDURES_KEY = "procedures"
INTERVAL_DAYS_KEY = "days_since_first_visit"

def flatten(l: List):
    return [item for sublist in l for item in sublist]

def patient_level_readmission_prediction(patient, time_window=30):
    """
    patient is a <pyhealth.data.Patient> object
    """
    samples = []

    # if the patient only has one visit, we drop it
    if len(patient) <= 2:
        return []

    sorted_visits = sorted(patient, key=lambda visit: visit.encounter_time)

    # step 1: define label
    idx_last_visit = len(sorted_visits)-1
    last_visit: Visit = sorted_visits[idx_last_visit]
    second_to_last_visit: Visit = sorted_visits[idx_last_visit - 1]
    first_visit: Visit = sorted_visits[0]

    time_diff = (last_visit.encounter_time - second_to_last_visit.encounter_time).days
    readmission_label = 1 if time_diff < time_window else 0

    # step 2: obtain features
    visits_conditions = []
    visits_procedures = []
    visits_intervals = []
    for idx, visit in enumerate(sorted_visits):
        if idx == len(sorted_visits) - 1: break # Don't include the last visit
        conditions = [c for c in visit.get_code_list(table="DIAGNOSES_ICD") if c in filtered_diag_codes]
        procedures = visit.get_code_list(table="PROCEDURES_ICD")
        time_diff_from_first_visit = (visit.encounter_time - first_visit.encounter_time).days

        if len(conditions) * len(procedures) == 0:
            continue

        visits_conditions.append(conditions)
        visits_procedures.append(procedures)
        visits_intervals.append([str(time_diff_from_first_visit)])

    unique_conditions = list(set(flatten(visits_conditions)))
    unique_procedures = list(set(flatten(visits_procedures)))

    # step 3: exclusion criteria
    if len(unique_conditions) * len(unique_procedures) == 0:
        return []

    # step 4: assemble the sample
    samples.append(
        {
            "patient_id": patient.patient_id,
            "visit_id": visit.visit_id,
            "conditions": visits_conditions,
            "procedures": visits_procedures,
            "intervals": visits_intervals,
            "label": readmission_label,
        }
    )
    return samples

In [191]:
# Create the task datasets
mimic3_dxtx = mimic3_ds.set_task(task_fn=patient_level_readmission_prediction)

Generating samples for patient_level_readmission_prediction: 100%|██████████| 46520/46520 [00:06<00:00, 7039.14it/s]


In [212]:
BATCH_SIZE = 32
train, val, test = split_by_patient(mimic3_dxtx, [0.8, 0.1, 0.1])

train_loader = get_dataloader(train, batch_size=BATCH_SIZE, shuffle=True)
val_loader = get_dataloader(val, batch_size=BATCH_SIZE, shuffle=False)
test_loader = get_dataloader(test, batch_size=BATCH_SIZE, shuffle=False)

In [232]:
# Define the models
VERY_BIG_NUMBER = 1e30
VERY_SMALL_NUMBER = 1e-30
VERY_POSITIVE_NUMBER = VERY_BIG_NUMBER
VERY_NEGATIVE_NUMBER = -VERY_BIG_NUMBER

class MaskDirection(Enum):
    FORWARD = 'forward'
    BACKWARD = 'backward'
    DIAGONAL = 'diagonal'
    NONE = 'none'

class MaskedLayerNorm(nn.Module):
    def __init__(self, normalized_shape: int):
        super().__init__()
        self.scale = nn.parameter.Parameter(torch.ones(normalized_shape, dtype=torch.float32))
        self.bias = nn.parameter.Parameter(torch.zeros(normalized_shape, dtype=torch.float32))
        self.normalized_shape = normalized_shape

    def forward(self, x: torch.Tensor, eps=1e-5):
        mean = torch.mean(x, dim=-1, keepdim=True)
        variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
        norm_x = (x - mean) * torch.rsqrt(variance + eps)
        return norm_x * self.scale + self.bias

class Flatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x: torch.Tensor, keep: int):
        fixed_shape = list(x.size())
        start = len(fixed_shape) - keep
        left = reduce(mul, [fixed_shape[i] or x.shape[i] for i in range(start)])
        out_shape = [left] + [fixed_shape[i] or x.shape[i] for i in range(start, len(fixed_shape))]
        return torch.reshape(x, out_shape)


class Unflatten(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, v: torch.Tensor, ref: torch.Tensor, embedding_dim):
        batch_size = ref.shape[0]
        n_visits = ref.shape[1]
        out = torch.reshape(v, [batch_size, n_visits, embedding_dim])
        return out


class AttentionPooling(nn.Module):
    def __init__(self, embedding_size: int):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(embedding_size, embedding_size),
            nn.ReLU(),
            nn.Linear(embedding_size, embedding_size)
        )

    def forward(self, inputs):
        x, mask = inputs
        x = self.fc(x)
        x[mask] = VERY_NEGATIVE_NUMBER
        soft = F.softmax(x, dim=1)
        x[mask] = 0
        attn_output = torch.sum(soft * x, 1)
        return attn_output


class MaskEnc(nn.Module):
    def __init__(
            self,
            embedding_dim: int,
            num_heads: int,
            dropout: float = 0.1,
            batch_first: bool = True,
            temporal_mask_direction: MaskDirection = MaskDirection.NONE,
    ):
        super().__init__()

        self.embedding_dim = embedding_dim
        self.temporal_mask_direction = temporal_mask_direction

        self.attention = nn.MultiheadAttention(
                embed_dim=embedding_dim,
                num_heads=num_heads,
                dropout=dropout,
                batch_first=batch_first
            )

        self.fc = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(embedding_dim, embedding_dim)
        )

        self.layer_norm1 = MaskedLayerNorm(embedding_dim)
        self.layer_norm2 = MaskedLayerNorm(embedding_dim)

    def forward(self, inputs):
        x, key_padding_mask = inputs
        numerical_key_padding_mask = torch.full(key_padding_mask.shape, VERY_NEGATIVE_NUMBER)
        numerical_key_padding_mask[~key_padding_mask] = 0
        attn_mask = self._make_temporal_mask(x.shape[1])

        attn_output, attn_output_weights = self.attention(x, x, x, key_padding_mask=numerical_key_padding_mask,
                                                          attn_mask=attn_mask)
        attn_output = torch.nan_to_num(attn_output, nan=0)
        attn_output = attn_output * (~key_padding_mask.unsqueeze(-1)).float()
        attn_output = self.layer_norm1(x + attn_output)
        out = self.fc(attn_output)
        out = out * (~key_padding_mask.unsqueeze(-1)).float()
        out = self.layer_norm2(out + attn_output)
        out = out * (~key_padding_mask.unsqueeze(-1)).float()

        return out, key_padding_mask

    def _make_temporal_mask(self, n: int) -> Optional[torch.Tensor]:
        if self.temporal_mask_direction == MaskDirection.NONE:
            return None
        if self.temporal_mask_direction == MaskDirection.FORWARD:
            return torch.tril(torch.full((n, n), VERY_NEGATIVE_NUMBER)).fill_diagonal_(0)
        if self.temporal_mask_direction == MaskDirection.BACKWARD:
            return torch.triu(torch.full((n, n), VERY_NEGATIVE_NUMBER)).fill_diagonal_(0)
        if self.temporal_mask_direction == MaskDirection.DIAGONAL:
            return torch.zeros(n, n).fill_diagonal_(VERY_NEGATIVE_NUMBER)


class BiteNet(nn.Module):
    def __init__(
            self,
            embedding_dim: int = 128,
            num_heads: int = 4,
            dropout: float = 0.1,
            batch_first: bool = True,
            n_mask_enc_layers: int = 2,
            use_procedures: bool = True,
            use_intervals: bool = True,
    ):
        super().__init__()

        self.use_intervals = use_intervals
        self.use_procedures = use_procedures
        self.embedding_dim = embedding_dim

        self.flatten = Flatten()
        self.unflatten = Unflatten()

        def _make_mask_enc_block(temporal_mask_direction: MaskDirection = MaskDirection.NONE):
            return MaskEnc(
                embedding_dim = embedding_dim,
                num_heads = num_heads,
                dropout = dropout,
                batch_first = batch_first,
                temporal_mask_direction = temporal_mask_direction,
            )

        self.code_attn = nn.Sequential()
        self.visit_attn_fw = nn.Sequential()
        self.visit_attn_bw = nn.Sequential()
        for _ in range(n_mask_enc_layers):
            self.code_attn.append(_make_mask_enc_block(MaskDirection.DIAGONAL))
            self.visit_attn_fw.append(_make_mask_enc_block(MaskDirection.FORWARD))
            self.visit_attn_bw.append(_make_mask_enc_block(MaskDirection.BACKWARD))

        # Attention pooling layers
        self.code_attn.append(AttentionPooling(embedding_dim))
        self.visit_attn_fw.append(AttentionPooling(embedding_dim))
        self.visit_attn_bw.append(AttentionPooling(embedding_dim))

        self.fc = nn.Sequential(
            nn.Linear(2*embedding_dim, embedding_dim),
            nn.ReLU()
        )

    def forward(
            self,
            embedded_codes: torch.Tensor,
            embedded_intervals: torch.Tensor,
            codes_mask: torch.Tensor,
            visits_mask: torch.Tensor,
    ) -> torch.Tensor:

        codes_mask = ~(codes_mask.bool())

        # input tensor, reshape 4 dimension to 3
        flattened_codes = self.flatten(embedded_codes, 2)

        # input mask, reshape 3 dimension to 2
        flattened_codes_mask = self.flatten(codes_mask, 1)

        code_attn = self.code_attn((flattened_codes, flattened_codes_mask))
        code_attn = self.unflatten(code_attn, embedded_codes, self.embedding_dim)

        if self.use_intervals:
            code_attn += embedded_intervals

        visits_mask = ~(visits_mask.bool())

        u_fw = self.visit_attn_fw((code_attn, visits_mask))
        u_bw = self.visit_attn_bw((code_attn, visits_mask))
        u_bi = torch.cat([u_fw, u_bw], dim=-1)

        s = self.fc(u_bi)
        return s

class PyHealthBiteNet(BaseModel):
    def __init__(
            self,
            dataset: SampleDataset,
            feature_keys: List[str],
            label_key: str,
            mode: str,
            embedding_dim: int = 128,
            n_mask_enc_layers: int = 2,
            use_intervals: bool = True,
            use_procedures: bool = True,
            num_heads: int = 4,
            dropout: float = 0.1,
            batch_first: bool = True,
            **kwargs
    ):
        super().__init__(dataset, feature_keys, label_key, mode)

        self.use_intervals = use_intervals
        self.use_procedures = use_procedures

        # Any BaseModel should have these attributes, as functions like add_feature_transform_layer uses them
        self.feat_tokenizers = {}
        self.embeddings = nn.ModuleDict()
        self.linear_layers = nn.ModuleDict()
        self.label_tokenizer = self.get_label_tokenizer()
        self.embedding_dim = embedding_dim

        # self.add_feature_transform_layer will create a transformation layer for each feature
        for feature_key in self.feature_keys:
            input_info = self.dataset.input_info[feature_key]
            self.add_feature_transform_layer(
                feature_key, input_info, special_tokens=["<pad>", "<unk>"]
            )

        # final output layer
        output_size = self.get_output_size(self.label_tokenizer)
        self.bite_net = BiteNet(
            embedding_dim = embedding_dim,
            num_heads = num_heads,
            dropout = dropout,
            batch_first = batch_first,
            use_intervals=use_intervals,
            use_procedures=use_procedures,
            n_mask_enc_layers=n_mask_enc_layers
        )

        self.fc = nn.Linear(self.embedding_dim, output_size)

    def forward(self, **kwargs) -> Dict[str, torch.Tensor]:

        embeddings = {}
        masks = {}
        for feature_key in self.feature_keys:
            input_info = self.dataset.input_info[feature_key]

            # each patient's feature is represented by [[code1, code2],[code3]]
            assert input_info["dim"] == 3 and input_info["type"] == str
            feature_vals = kwargs[feature_key]

            x = self.feat_tokenizers[feature_key].batch_encode_3d(feature_vals, truncation=(False, False))
            x = torch.tensor(x, dtype=torch.long, device=self.device)
            pad_idx = self.feat_tokenizers[feature_key].vocabulary("<pad>")
            #create the mask
            mask = (x != pad_idx).long()
            embeds = self.embeddings[feature_key](x)
            embeddings[feature_key] = embeds
            masks[feature_key] = mask

        embedded_codes = embeddings['conditions']
        codes_mask = masks['conditions']
        if self.use_procedures:
            embedded_codes = torch.cat((embedded_codes, embeddings['procedures']), dim=2)
            codes_mask = torch.cat((codes_mask, masks['procedures']), dim=2)

        output = self.bite_net(embedded_codes, embeddings['intervals'].squeeze(2), codes_mask, masks['intervals'].squeeze(-1))
        logits = self.fc(output)

        # obtain y_true, loss, y_prob
        y_true = self.prepare_labels(kwargs[self.label_key], self.label_tokenizer)
        loss = self.get_loss_function()(logits, y_true)
        y_prob = self.prepare_y_prob(logits)

        return {"loss": loss, "y_prob": y_prob, "y_true": y_true}

model_dxtx = PyHealthBiteNet(
    dataset = mimic3_dxtx,
    feature_keys = ['procedures', 'conditions', 'intervals'],
    label_key = "label",
    mode = "binary",
    embedding_dim=128
)

data = next(iter(train_loader))
model_dxtx(**data)

{'loss': tensor(0.7056, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'y_prob': tensor([[0.5057],
         [0.5280],
         [0.5071],
         [0.5181],
         [0.5144],
         [0.5170],
         [0.5181],
         [0.5319],
         [0.5085],
         [0.5168],
         [0.5125],
         [0.5083],
         [0.5068],
         [0.5052],
         [0.5193],
         [0.5168],
         [0.5115],
         [0.5075],
         [0.5194],
         [0.5055],
         [0.5169],
         [0.5234],
         [0.5118],
         [0.5213],
         [0.5165],
         [0.5081],
         [0.5157],
         [0.5042],
         [0.5146],
         [0.5112],
         [0.5214],
         [0.5222]], grad_fn=<SigmoidBackward0>),
 'y_true': tensor([[1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [1.],
         [0.],
         [1.],
         [1.],
         [0.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.]

In [233]:
model_dxtx = PyHealthBiteNet(
    dataset = mimic3_dxtx,
    feature_keys = ["conditions", "procedures", "intervals"],
    label_key = "label",
    mode = "binary",
    use_intervals=False
)

In [234]:
trainer_dxtx = Trainer(model=model_dxtx)
trainer_dxtx.train(
    train_dataloader=train_loader,
    val_dataloader=val_loader,
    epochs=10,
    monitor="pr_auc",
    optimizer_class=torch.optim.RMSprop
)

PyHealthBiteNet(
  (embeddings): ModuleDict(
    (conditions): Embedding(2834, 128, padding_idx=0)
    (procedures): Embedding(1061, 128, padding_idx=0)
    (intervals): Embedding(1758, 128, padding_idx=0)
  )
  (linear_layers): ModuleDict()
  (bite_net): BiteNet(
    (flatten): Flatten()
    (unflatten): Unflatten()
    (code_attn): Sequential(
      (0): MaskEnc(
        (attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
        )
        (fc): Sequential(
          (0): Linear(in_features=128, out_features=128, bias=True)
          (1): ReLU()
          (2): Dropout(p=0.1, inplace=False)
          (3): Linear(in_features=128, out_features=128, bias=True)
        )
        (layer_norm1): MaskedLayerNorm()
        (layer_norm2): MaskedLayerNorm()
      )
      (1): MaskEnc(
        (attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bi

Epoch 0 / 10:   0%|          | 0/58 [00:00<?, ?it/s]

--- Train epoch-0, step-58 ---
loss: 0.5553
Evaluation:   0%|          | 0/8 [00:00<?, ?it/s]/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(
Evaluation: 100%|██████████| 8/8 [00:00<00:00, 19.95it/s]
--- Eval epoch-0, step-58 ---
pr_auc: 0.2282
roc_auc: 0.4993
f1: 0.0000
loss: 0.5058
New best pr_auc score (0.2282) at epoch-0, step-58



Epoch 1 / 10:   0%|          | 0/58 [00:00<?, ?it/s]

--- Train epoch-1, step-116 ---
loss: 0.5052
Evaluation:   0%|          | 0/8 [00:00<?, ?it/s]/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(
Evaluation: 100%|██████████| 8/8 [00:00<00:00, 19.42it/s]
--- Eval epoch-1, step-116 ---
pr_auc: 0.2356
roc_auc: 0.4927
f1: 0.0000
loss: 0.5048
New best pr_auc score (0.2356) at epoch-1, step-116



...

--- Train epoch-6, step-406 ---
loss: 0.5051
Evaluation:   0%|          | 0/8 [00:00<?, ?it/s]/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(
Evaluation: 100%|██████████| 8/8 [00:00<00:00, 17.95it/s]
--- Eval epoch-6, step-406 ---
pr_auc: 0.2133
roc_auc: 0.4561
f1: 0.0000
loss: 0.5052



Epoch 7 / 10:   0%|          | 0/58 [00:00<?, ?it/s]

--- Train epoch-7, step-464 ---
loss: 0.5050
Evaluation:   0%|          | 0/8 [00:00<?, ?it/s]/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(
Evaluation: 100%|██████████| 8/8 [00:00<00:00, 18.73it/s]
--- Eval epoch-7, step-464 ---
pr_auc: 0.2191
roc_auc: 0.5114
f1: 0.0000
loss: 0.5057



Epoch 8 / 10:   0%|          | 0/58 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [226]:
# option 1: use our built-in evaluation metric
score_dxtx = trainer_dxtx.evaluate(test_loader)
print (score_dxtx)

# option 2: use our pyhealth.metrics to evaluate
y_true_dxtx, y_prob_dxtx, loss_dxtx = trainer_dxtx.inference(test_loader)
binary_metrics_fn(y_true_dxtx, y_prob_dxtx, metrics=["pr_auc"])

Evaluation:   0%|          | 0/8 [00:00<?, ?it/s]/Users/cyg1122/PycharmProjects/dl4h-final-project/venv/lib/python3.8/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(
Evaluation: 100%|██████████| 8/8 [00:00<00:00, 17.61it/s]


ValueError: Input contains NaN.

In [235]:
data = next(iter(train_loader))
model_dxtx(**data)

{'loss': tensor(0.7193, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'y_prob': tensor([[0.1801],
         [0.1800],
         [0.1802],
         [0.1802],
         [0.1799],
         [0.1804],
         [0.1810],
         [0.1804],
         [0.1809],
         [0.1815],
         [0.1802],
         [0.1802],
         [0.1799],
         [0.1803],
         [0.1800],
         [0.1801],
         [0.1804],
         [0.1800],
         [0.1797],
         [0.1806],
         [0.1809],
         [0.1803],
         [0.1805],
         [0.1796],
         [0.1805],
         [0.1805],
         [0.1809],
         [0.1813],
         [0.1820],
         [0.1798],
         [0.1801],
         [0.1811]], grad_fn=<SigmoidBackward0>),
 'y_true': tensor([[0.],
         [1.],
         [0.],
         [0.],
         [1.],
         [0.],
         [1.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [1.],
         [1.],
         [0.],
         [1.],
         [0.],
         [1.]